In [1]:
import pandas as pd
import numpy as np
import nba_api.stats.endpoints as nba
import nba_api.stats.static.players as players
import nba_api.stats.static.teams as teams
import networkx as nx

In [49]:
df = pd.read_csv('data/active_player_info_team.csv')
df

,index,id,Full_name,Firstname,Surname,Active,team_name,team_id
0,0,1630173,Precious Achiuwa,Precious,Achiuwa,True,Knicks,1610612752
1,1,203500,Steven Adams,Steven,Adams,True,Rockets,1610612745
2,2,1628389,Bam Adebayo,Bam,Adebayo,True,Heat,1610612748
3,3,1630534,Ochai Agbaji,Ochai,Agbaji,True,Raptors,1610612761
4,4,1631231,James Akinjo,James,Akinjo,True,NaN,0
...,...,...,...,...,...,...,...,...
610,610,203469,Cody Zeller,Cody,Zeller,True,Hawks,1610612737
611,611,1627826,Ivica Zubac,Ivica,Zubac,True,Clippers,1610612746
612,612,1641783,Tristan da Silva,Tristan,da Silva,True,Magic,1610612753
613,613,1628427,Vlatko Čančar,Vlatko,Čančar,True,Nuggets,1610612743


In [50]:
passing_stats_df = pd.read_csv('data/NBA_pass_data_2324_totals_regularseason.csv')
passing_stats_df

,Unnamed: 0,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_TO,PASS_TEAMMATE_PLAYER_ID,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,0,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Banton, Dalano",1630625,...,2,2,4,0.500,1,2,0.5,1,2,0.500
1,1,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Hauser, Sam",1630573,...,39,40,82,0.488,6,12,0.5,34,70,0.486
2,2,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Springer, Jaden",1630531,...,0,0,1,0.000,0,0,0.0,0,1,0.000
3,3,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Tillman, Xavier",1630214,...,2,2,7,0.286,0,2,0.0,2,5,0.400
4,4,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Stevens, Lamar",1630205,...,1,1,3,0.333,0,0,0.0,1,3,0.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,3,201959,"Gibson, Taj",Detroit Pistons,1610612765,DET,made,4,"Flynn, Malachi",1630201,...,0,0,4,0.000,0,2,0.0,0,2,0.000
8181,4,201959,"Gibson, Taj",Detroit Pistons,1610612765,DET,made,4,"Wiseman, James",1630164,...,0,0,0,0.000,0,0,0.0,0,0,0.000
8182,5,201959,"Gibson, Taj",Detroit Pistons,1610612765,DET,made,4,"Nowell, Jaylen",1629669,...,0,0,1,0.000,0,0,0.0,0,1,0.000
8183,6,201959,"Gibson, Taj",Detroit Pistons,1610612765,DET,made,4,"Brown Jr., Troy",1628972,...,1,1,2,0.500,1,2,0.5,0,0,0.000


In [51]:
df_total_score = pd.read_csv('data/nba_players_total_points_2023_24.csv')
df_total_score

,Player Name,Player ID,Total Points
0,"Tatum, Jayson",1628369,1987
1,"Holiday, Jrue",201950,860
2,"Brown, Jaylen",1627759,0
3,"Porziņģis, Kristaps",204001,1145
4,"White, Derrick",1628401,1107
...,...,...,...
517,"Sasser, Marcus",1631204,586
518,"Boeheim, Buddy",1631205,34
519,"Stewart, Isaiah",1630191,503
520,"Fournier, Evan",203095,220


Make the graph of the player total scoring and the assists that they have

In [6]:
G = nx.DiGraph()
G.add_node('GOAL', attr='Goal')

In [7]:
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)

In [8]:
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['PASS']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight) # To avoid having nodes that don't have any edges we give each edges a weight of at least 1, to ensure that the communities remain the same

In [10]:
for index, row in df_total_score.iterrows():
    edge = (row['Player Name'], 'GOAL')
    weight = row['Total Points']
    G.add_edge(edge[0], edge[1], weight=weight) # add the number of points from a player to the GOAL        

In [11]:
# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot1_pass_score.gexf")

Make the graph of the player total scoring and the assists that they have without the goal node

In [5]:
G = nx.DiGraph()
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['PASS']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1],
                   weight=weight) 
# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot2_pass.gexf")

Make the graph with the number of passed between each player and the number of made attempts based on the number of total attempts (value normalized based on the number of game played)

In [52]:
G = nx.DiGraph()
G.add_node('GOAL', attr='Goal')

In [53]:
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)

In [54]:
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['FREQUENCY']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight)

In [55]:
player_counts = passing_stats_df['PLAYER_NAME_LAST_FIRST'].value_counts()

for index, row in passing_stats_df.iterrows():
    if row['FGA'] == 0 and row['FG2A'] == 0 and row['FG3A'] == 0:
        player_name = row['PLAYER_NAME_LAST_FIRST']
        player_counts[player_name] -= 1
        

In [56]:
for index, row in passing_stats_df.iterrows():
    count = player_counts.get(row['PLAYER_NAME_LAST_FIRST'],0)
    edge = (row['PLAYER_NAME_LAST_FIRST'], 'GOAL')
    # Determine the number of non-zero attempt columns
    attempts = [row['FGA'], row['FG2A'], row['FG3A']]
    non_zero_attempts = sum(1 for attempt in attempts if attempt != 0)
    
    # Avoid division by zero in case all attempts are zero
    if non_zero_attempts > 0:
        weight = (row['FG_PCT'] + row['FG2_PCT'] + row['FG3_PCT']) / non_zero_attempts
        weight /= count
    else:
        weight = 0  # Assign 0 weight if no attempts
    
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight)

In [57]:
# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot3_freq_attempts.gexf")

Make the graph with the number of passed between each player and the number of made attempts based on the number of total attempts (value normalized based on the number of game played) without the goal

In [38]:
G = nx.DiGraph()
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['FREQUENCY']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight)

# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot4_freq.gexf")

Make the graph of the player total scoring and the assists that they have (Specific team Boston)

In [47]:
passing_stats_df = pd.read_csv('data/NBA_pass_data_2324_totals_regularseason.csv')
passing_stats_df = passing_stats_df.loc[passing_stats_df["TEAM_ID"] == 1610612738] #Boston
passing_stats_df

,Unnamed: 0,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_TO,PASS_TEAMMATE_PLAYER_ID,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,0,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Banton, Dalano",1630625,...,2,2,4,0.500,1,2,0.500,1,2,0.500
1,1,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Hauser, Sam",1630573,...,39,40,82,0.488,6,12,0.500,34,70,0.486
2,2,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Springer, Jaden",1630531,...,0,0,1,0.000,0,0,0.000,0,1,0.000
3,3,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Tillman, Xavier",1630214,...,2,2,7,0.286,0,2,0.000,2,5,0.400
4,4,1628369,"Tatum, Jayson",Boston Celtics,1610612738,BOS,made,73,"Stevens, Lamar",1630205,...,1,1,3,0.333,0,0,0.000,1,3,0.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,12,1629674,"Queta, Neemias",Boston Celtics,1610612738,BOS,made,28,"White, Derrick",1628401,...,1,3,10,0.300,2,6,0.333,1,4,0.250
240,13,1629674,"Queta, Neemias",Boston Celtics,1610612738,BOS,made,28,"Tatum, Jayson",1628369,...,1,3,9,0.333,1,5,0.200,2,4,0.500
241,14,1629674,"Queta, Neemias",Boston Celtics,1610612738,BOS,made,28,"Brown, Jaylen",1627759,...,0,1,1,1.000,0,0,0.000,1,1,1.000
242,15,1629674,"Queta, Neemias",Boston Celtics,1610612738,BOS,made,28,"Holiday, Jrue",201950,...,1,3,5,0.600,2,4,0.500,1,1,1.000


In [48]:
G = nx.DiGraph()
G.add_node('GOAL', attr='Goal')
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['AST']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1],weight=weight) 
        
for index, row in df_total_score.iterrows():
    edge = (row['Player Name'], 'GOAL')
    weight = row['Total Points']
    if G.has_node(row['Player Name']):
        G.add_edge(edge[0], edge[1], weight=weight)  # add the number of points from a player to the GOAL        
# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot5_Boston_assist_score.gexf")

Make the graph with the number of passed between each player and the number of made attempts based on the number of total attempts (value normalized based on the number of game played) (1610612747 Lakers)

In [58]:
passing_stats_df = pd.read_csv('data/NBA_pass_data_2324_totals_regularseason.csv')
passing_stats_df = passing_stats_df.loc[passing_stats_df["TEAM_ID"] == 1610612747] #Lakers

In [59]:
G = nx.DiGraph()
G.add_node('GOAL', attr='Goal')
for index, row in passing_stats_df.iterrows():
    player = row['PLAYER_NAME_LAST_FIRST']
    player2 = row['PASS_TO']
    team = row['TEAM_NAME']
    if not G.has_node(player):
        G.add_node(player, attr=team)
    if not G.has_node(player2):
        G.add_node(player2, attr=team)
for index, row in passing_stats_df.iterrows():
    edge = (row['PLAYER_NAME_LAST_FIRST'], row['PASS_TO'])
    weight = row['FREQUENCY']
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight)
player_counts = passing_stats_df['PLAYER_NAME_LAST_FIRST'].value_counts()

for index, row in passing_stats_df.iterrows():
    if row['FGA'] == 0 and row['FG2A'] == 0 and row['FG3A'] == 0:
        player_name = row['PLAYER_NAME_LAST_FIRST']
        player_counts[player_name] -= 1

for index, row in passing_stats_df.iterrows():
    count = player_counts.get(row['PLAYER_NAME_LAST_FIRST'],0)
    edge = (row['PLAYER_NAME_LAST_FIRST'], 'GOAL')
    # Determine the number of non-zero attempt columns
    attempts = [row['FGA'], row['FG2A'], row['FG3A']]
    non_zero_attempts = sum(1 for attempt in attempts if attempt != 0)
    
    # Avoid division by zero in case all attempts are zero
    if non_zero_attempts > 0:
        weight = (row['FG_PCT'] + row['FG2_PCT'] + row['FG3_PCT']) / non_zero_attempts
        weight /= count
    else:
        weight = 0  # Assign 0 weight if no attempts
    
    if G.has_edge(*edge):
        # If the edge exists, add to the current weight
        G[edge[0]][edge[1]]['weight'] += weight
    else:
        G.add_edge(edge[0], edge[1], weight=weight)   
# Export the graph as .gexf to load it in Gephi
nx.write_gexf(G, "plot6_Lakers_passes_attempts.gexf")

In [61]:
# Calculate and print in-degree and sum of incoming edge weights for each node
print("In-degree and sum of incoming edge weights for each node:")
for node in G.nodes():
    in_degree = G.in_degree(node)
    # Get the sum of weights for all incoming edges
    in_edges = G.in_edges(node, data=True)
    total_incoming_weight = sum(edge_data['weight'] for _, _, edge_data in in_edges)
    print(f"{node}: In-degree = {in_degree}, Total incoming weight = {total_incoming_weight}")

# Example: Specific node, e.g., 'GOAL'
goal_in_degree = G.in_degree('GOAL')
goal_in_edges = G.in_edges('GOAL', data=True)
goal_total_weight = sum(edge_data['weight'] for _, _, edge_data in goal_in_edges)
print(f"'GOAL': In-degree = {goal_in_degree}, Total incoming weight = {goal_total_weight}")


In-degree and sum of incoming edge weights for each node:
GOAL: In-degree = 18, Total incoming weight = 6.557841185448537
Hood-Schifino, Jalen: In-degree = 15, Total incoming weight = 0.8700000000000001
Hodge, D'Moi: In-degree = 9, Total incoming weight = 0.162
Fudge, Alex: In-degree = 4, Total incoming weight = 0.07
Lewis, Maxwell: In-degree = 11, Total incoming weight = 0.454
Christie, Max: In-degree = 18, Total incoming weight = 1.5170000000000001
Castleton, Colin: In-degree = 9, Total incoming weight = 0.151
Reaves, Austin: In-degree = 15, Total incoming weight = 2.5220000000000002
Mays, Skylar: In-degree = 14, Total incoming weight = 0.552
Windler, Dylan: In-degree = 8, Total incoming weight = 0.163
Hayes, Jaxson: In-degree = 16, Total incoming weight = 0.6370000000000002
Vanderbilt, Jarred: In-degree = 13, Total incoming weight = 0.19400000000000006
Giles III, Harry: In-degree = 5, Total incoming weight = 0.07500000000000001
Wood, Christian: In-degree = 17, Total incoming weight 

In [62]:
# Calculate the total weight of all incoming edges excluding 'GOAL'
total_weight_excluding_goal = 0
for node in G.nodes():
    if node != 'GOAL':
        in_edges = G.in_edges(node, data=True)
        total_weight_excluding_goal += sum(edge_data['weight'] for _, _, edge_data in in_edges)

print(f"Total incoming weight excluding 'GOAL': {total_weight_excluding_goal}")

# Calculate normalized incoming weights for each node (excluding 'GOAL')
print("Normalized total incoming weights for each node (excluding 'GOAL'):")
for node in G.nodes():
    if node != 'GOAL':
        in_edges = G.in_edges(node, data=True)
        total_incoming_weight = sum(edge_data['weight'] for _, _, edge_data in in_edges)
        normalized_weight = total_incoming_weight / total_weight_excluding_goal if total_weight_excluding_goal > 0 else 0
        print(f"{node}: Normalized incoming weight = {normalized_weight:.4f}")


Total incoming weight excluding 'GOAL': 18.000000000000004
Normalized total incoming weights for each node (excluding 'GOAL'):
Hood-Schifino, Jalen: Normalized incoming weight = 0.0483
Hodge, D'Moi: Normalized incoming weight = 0.0090
Fudge, Alex: Normalized incoming weight = 0.0039
Lewis, Maxwell: Normalized incoming weight = 0.0252
Christie, Max: Normalized incoming weight = 0.0843
Castleton, Colin: Normalized incoming weight = 0.0084
Reaves, Austin: Normalized incoming weight = 0.1401
Mays, Skylar: Normalized incoming weight = 0.0307
Windler, Dylan: Normalized incoming weight = 0.0091
Hayes, Jaxson: Normalized incoming weight = 0.0354
Vanderbilt, Jarred: Normalized incoming weight = 0.0108
Giles III, Harry: Normalized incoming weight = 0.0042
Wood, Christian: Normalized incoming weight = 0.0364
Dinwiddie, Spencer: Normalized incoming weight = 0.0267
Davis, Anthony: Normalized incoming weight = 0.0967
James, LeBron: Normalized incoming weight = 0.1508
Russell, D'Angelo: Normalized in